In [21]:
from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np

In [22]:
%matplotlib inline
def show(img):
    npimg = img.numpy()
    npimg = npimg-np.amin(npimg)
    npimg = npimg/np.amax(npimg)
    plt.figure(figsize=(11,11))
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.axis("off")
        
    plt.show()

In [24]:
# pip ins# We will save the results here
outf="./dcgan_results"

In [26]:
#manualSeed = random.randint(1, 10000)
manualSeed = 4532
print("Random Seed: ", manualSeed)
random.seed(manualSeed)

torch.manual_seed(manualSeed)
cudnn.benchmark = True

use_cuda=True
if use_cuda:
    torch.cuda.manual_seed_all(manualSeed)

if torch.cuda.is_available() and not use_cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

Random Seed:  4532


In [46]:
from os import listdir
from os.path import isfile, join
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
listdir('/Users/cedriczhao/Documents/GitHub/dsc160-code/dsc160-final-404-not-found-1/data/sp')

['resized_image22.jpg',
 'resized_image36.jpg',
 'resized_image148.jpg',
 'resized_image149.jpg',
 'resized_image37.jpg',
 'resized_image23.jpg',
 'resized_image35.jpg',
 'resized_image21.jpg',
 'resized_image20.jpg',
 'resized_image34.jpg',
 'resized_image8.jpg',
 'resized_image18.jpg',
 'resized_image30.jpg',
 'resized_image24.jpg',
 'resized_image25.jpg',
 'resized_image31.jpg',
 'resized_image19.jpg',
 'resized_image9.jpg',
 'resized_image27.jpg',
 'resized_image33.jpg',
 'resized_image32.jpg',
 'resized_image26.jpg',
 'resized_image82.jpg',
 'resized_image96.jpg',
 'resized_image41.jpg',
 'resized_image55.jpg',
 'resized_image69.jpg',
 'resized_image117.jpg',
 'resized_image103.jpg',
 'resized_image102.jpg',
 'resized_image116.jpg',
 'resized_image68.jpg',
 'resized_image54.jpg',
 'resized_image40.jpg',
 'resized_image97.jpg',
 'resized_image83.jpg',
 'resized_image95.jpg',
 'resized_image81.jpg',
 'resized_image56.jpg',
 'resized_image42.jpg',
 'resized_image100.jpg',
 'resized_i

In [44]:
pwd

'/Users/cedriczhao/Documents/GitHub/dsc160-code/dsc160-final-404-not-found-1/code'

In [48]:
imageSize = 64 # square images for now!
dataroot = "/Users/cedriczhao/Documents/GitHub/dsc160-code/dsc160-final-404-not-found-1/data/superheroes"
#dataroot = "./geom_pics_dataset"

# folder dataset
dataset = dset.ImageFolder(root=dataroot,
                               transform=transforms.Compose([
                                   transforms.Scale(imageSize),
                                   transforms.CenterCrop(imageSize),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]))

RuntimeError: Found 0 files in subfolders of: /Users/cedriczhao/Documents/GitHub/dsc160-code/dsc160-final-404-not-found-1/data/superheroes
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif,.tiff,.webp